# Step 3
In step three we loop through the cropped GeoJson files describing the building and road outlines seperately, and then generate a Json for each one which describes what blocks are in each Minecraft chunk. These Json files are what will be used for the populator plugin

## Imports:

In [1]:
import geopandas
from terrapyconvert import from_geo

import numpy as np
import math
from typing import List, Tuple
import json
from shapely.ops import polygonize, unary_union
from shapely.geometry import Polygon, Point, LineString

Loading conformal correction data...
Loaded 33153 conformal correction points


## Bresenham with 3D points algorithm:

In [2]:
from typing import List, Tuple


def bresenham3d(x1: int, y1: int, z1: int, x2: int, y2: int, z2: int):
    list_of_points: List[Tuple[int, int, int]] = [(x1, y1, z1)]

    dx = abs(x2 - x1)
    dy = abs(y2 - y1)
    dz = abs(z2 - z1)
    if x2 > x1:
        xs = 1
    else:
        xs = -1
    if y2 > y1:
        ys = 1
    else:
        ys = -1
    if z2 > z1:
        zs = 1
    else:
        zs = -1

    # Driving axis is X-axis
    if dx >= dy and dx >= dz:
        p1 = 2 * dy - dx
        p2 = 2 * dz - dx
        while x1 != x2:
            x1 += xs
            if p1 >= 0:
                y1 += ys
                p1 -= 2 * dx
            if p2 >= 0:
                z1 += zs
                p2 -= 2 * dx
            p1 += 2 * dy
            p2 += 2 * dz
            list_of_points.append((x1, y1, z1))

    # Driving axis is Y-axis
    elif dy >= dx and dy >= dz:
        p1 = 2 * dx - dy
        p2 = 2 * dz - dy
        while y1 != y2:
            y1 += ys
            if p1 >= 0:
                x1 += xs
                p1 -= 2 * dy
            if p2 >= 0:
                z1 += zs
                p2 -= 2 * dy
            p1 += 2 * dx
            p2 += 2 * dz
            list_of_points.append((x1, y1, z1))

    # Driving axis is Z-axis
    else:
        p1 = 2 * dy - dz
        p2 = 2 * dx - dz
        while z1 != z2:
            z1 += zs
            if p1 >= 0:
                y1 += ys
                p1 -= 2 * dz
            if p2 >= 0:
                x1 += xs
                p2 -= 2 * dz
            p1 += 2 * dy
            p2 += 2 * dx
            list_of_points.append((x1, y1, z1))
    return list_of_points

## Calculate blocks for polygons

### Helper functions

In [3]:
def polygon_area(poly):
    a = 0.0
    n = len(poly)
    for i in range(n):
        x1, z1 = poly[i]
        x2, z2 = poly[(i + 1) % n]
        a += x1 * z2 - x2 * z1
    return a * 0.5


def cross(a, b, c):
    return (b[0] - a[0]) * (c[1] - a[1]) - (b[1] - a[1]) * (c[0] - a[0])


def is_convex(a, b, c, ccw):
    if ccw:
        return cross(a, b, c) > 1e-12
    else:
        return cross(a, b, c) < -1e-12


def point_in_triangle_bary(a, b, c, p):
    A = np.array([[a[0], b[0], c[0]], [a[1], b[1], c[1]], [1.0, 1.0, 1.0]])
    B = np.array([p[0], p[1], 1.0])
    try:
        w = np.linalg.solve(A, B)
    except np.linalg.LinAlgError:
        return None
    return tuple(w)


def point_in_triangle(a, b, c, p, eps=-1e-9):
    bary = point_in_triangle_bary(a, b, c, p)
    if bary is None:
        return False
    u, v, w = bary
    return (u >= eps) and (v >= eps) and (w >= eps)


def ear_clip_triangulate(poly):
    n = len(poly)
    if n < 3:
        return []
    V = list(range(n))
    triangles = []
    ccw = polygon_area(poly) > 0
    safety = 0
    while len(V) > 3 and safety < n * n:
        ear_found = False
        for vi in range(len(V)):
            i_prev = V[(vi - 1) % len(V)]
            i_curr = V[vi]
            i_next = V[(vi + 1) % len(V)]
            a = poly[i_prev]
            b = poly[i_curr]
            c = poly[i_next]
            if not is_convex(a, b, c, ccw):
                continue
            any_inside = False
            for other in V:
                if other in (i_prev, i_curr, i_next):
                    continue
                if point_in_triangle(a, b, c, poly[other]):
                    any_inside = True
                    break
            if any_inside:
                continue
            triangles.append((i_prev, i_curr, i_next))
            V.pop(vi)
            ear_found = True
            break
        if not ear_found:
            safety += 1
            break
    if len(V) == 3:
        triangles.append((V[0], V[1], V[2]))
    return triangles


# ----------------------
# Bresenham line (2D XZ)
# ----------------------
def bresenham_line(x0, z0, x1, z1):
    points = []
    dx = abs(x1 - x0)
    dz = abs(z1 - z0)
    sx = 1 if x0 < x1 else -1
    sz = 1 if z0 < z1 else -1
    err = dx - dz
    while True:
        points.append((x0, z0))
        if x0 == x1 and z0 == z1:
            break
        e2 = 2 * err
        if e2 > -dz:
            err -= dz
            x0 += sx
        if e2 < dx:
            err += dx
            z0 += sz
    return points

def ray_cast_even_odd(point, ring):
    """Return True if point is inside the (possibly self-intersecting) ring by even-odd rule.
       ring: list[(x,z)] closed or open."""
    x, z = point
    n = len(ring)
    inside = False
    for i in range(n):
        x1, z1 = ring[i]
        x2, z2 = ring[(i + 1) % n]
        # Check if edge crosses horizontal ray to +inf at z
        # Ignore horizontal edges by strict inequality on z
        if ((z1 > z) != (z2 > z)):
            # Solve for x at intersection
            xin = x1 + (x2 - x1) * (z - z1) / (z2 - z1)
            if xin > x:
                inside = not inside
    return inside

def point_on_segment_param(px, pz, x1, z1, x2, z2, tol=1e-7):
    """Return param t in [0,1] if (px,pz) lies on segment (x1,z1)-(x2,z2) within tol, else None."""
    vx, vz = x2 - x1, z2 - z1
    wx, wz = px - x1, pz - z1
    seglen2 = vx*vx + vz*vz
    if seglen2 == 0.0:
        # Degenerate segment; treat as a point
        d2 = (px - x1)**2 + (pz - z1)**2
        return 0.0 if d2 <= tol*tol else None
    # Projection parameter
    t = (wx*vx + wz*vz) / seglen2
    if t < -tol or t > 1.0 + tol:
        return None
    # Check perpendicular distance to the infinite line
    # distance^2 = |w - t*v|^2
    dx = wx - t*vx
    dz = wz - t*vz
    if dx*dx + dz*dz <= tol*tol:
        # Clamp t to [0,1]
        return max(0.0, min(1.0, t))
    return None

def build_segments_with_y(poly3):
    """Return list of segments with y attached: [(x1,z1,y1,x2,z2,y2), ...]"""
    segs = []
    m = len(poly3)
    for i in range(m):
        x1, y1, z1 = poly3[i]
        x2, y2, z2 = poly3[(i + 1) % m]
        segs.append((x1, z1, y1, x2, z2, y2))
    return segs

def interpolate_y_at_point(px, pz, segs, tol=1e-6):
    """Find the original outline segment that contains (px,pz) and interpolate Y."""
    for (x1, z1, y1, x2, z2, y2) in segs:
        t = point_on_segment_param(px, pz, x1, z1, x2, z2, tol=tol)
        if t is not None:
            return (1.0 - t) * y1 + t * y2
    # Fallback: if no segment matched (numerical quirks), just return average Y across outline
    if segs:
        avg_y = sum((s[2] + s[5]) * 0.5 for s in segs) / len(segs)
        return avg_y
    return 0.0

## Main functions
Ear clipping: https://en.wikipedia.org/wiki/Polygon_triangulation

Barycentric coordinates to interpolate y coordinates: https://en.wikipedia.org/wiki/Barycentric_coordinate_system

In [4]:
def get_polygon_points(polygon_3d):
    # Ensure no trailing duplicate
    if len(polygon_3d) > 1 and polygon_3d[0] == polygon_3d[-1]:
        poly3 = polygon_3d[:-1]
    else:
        poly3 = polygon_3d[:]

    # Project to XZ for footprint
    poly2 = [(p[0], p[2]) for p in poly3]
    poly_shape = Polygon(poly2)
    if not poly_shape.is_valid:
        print("Using alternative function")
        return get_polygon_points_v2(polygon_3d)
        #raise ValueError("Projected 2D polygon is invalid (self-intersecting).")

    # Triangulate in 2D
    tri_indices = ear_clip_triangulate(poly2)
    if len(tri_indices) == 0:
        raise ValueError("Triangulation failed. Check polygon geometry.")

    # Bounding box to iterate integer block positions
    min_x = int(math.floor(min(x for x, z in poly2)))
    max_x = int(math.ceil(max(x for x, z in poly2)))
    min_z = int(math.floor(min(z for x, z in poly2)))
    max_z = int(math.ceil(max(z for x, z in poly2)))

    filled_blocks = []  # interior blocks
    missed = 0

    for X in range(min_x, max_x + 1):
        for Z in range(min_z, max_z + 1):
            sample = (X,Z)
            if not poly_shape.contains(Point(sample)):
                continue
            assigned = False
            for (i, j, k) in tri_indices:
                a2, b2, c2 = poly2[i], poly2[j], poly2[k]
                bary = point_in_triangle_bary(a2, b2, c2, sample)
                if bary is None:
                    continue
                u, v, w = bary
                if (u >= -1e-9) and (v >= -1e-9) and (w >= -1e-9):
                    ya, yb, yc = poly3[i][1], poly3[j][1], poly3[k][1]
                    y = u * ya + v * yb + w * yc
                    filled_blocks.append((X, int(round(y)), Z))
                    assigned = True
                    break
            if not assigned:
                best = None
                best_minw = -1.0
                for (i, j, k) in tri_indices:
                    a2, b2, c2 = poly2[i], poly2[j], poly2[k]
                    bary = point_in_triangle_bary(a2, b2, c2, sample)
                    if bary is None:
                        continue
                    u, v, w = bary
                    minw = min(u, v, w)
                    if minw > best_minw:
                        best_minw = minw
                        best = (i, j, k, bary)
                if best is not None:
                    i, j, k, bary = best
                    u, v, w = bary
                    ya, yb, yc = poly3[i][1], poly3[j][1], poly3[k][1]
                    y = u * ya + v * yb + w * yc
                    filled_blocks.append((X, int(round(y)), Z))
                else:
                    avg_y = sum(p[1] for p in poly3) / len(poly3)
                    filled_blocks.append((X, int(round(avg_y)), Z))
                    missed += 1

    # ----------------------
    # Edge blocks with Bresenham
    # ----------------------
    edge_blocks = []
    for i in range(len(poly3)):
        x0, y0, z0 = poly3[i]
        x1, y1, z1 = poly3[(i + 1) % len(poly3)]
        edge_points = bresenham_line(int(round(x0)), int(round(z0)),
                                     int(round(x1)), int(round(z1)))
        n = len(edge_points)
        for idx, (x, z) in enumerate(edge_points):
            t = idx / max(1, n - 1)
            y = (1 - t) * y0 + t * y1
            edge_blocks.append((x, int(round(y)), z))

    # ----------------------
    # Combine filled + edge blocks
    # ----------------------
    all_blocks = set(filled_blocks) | set(edge_blocks)

    return all_blocks

def get_polygon_points_v2(polygon_3d):
    # Remove trailing duplicate
    if len(polygon_3d) > 1 and polygon_3d[0] == polygon_3d[-1]:
        poly3 = polygon_3d[:-1]
    else:
        poly3 = polygon_3d[:]

    # Project to XZ for footprint
    poly2 = [(p[0], p[2]) for p in poly3]

    # Attempt normal path first (simple polygon)
    poly_shape = Polygon(poly2)
    if poly_shape.is_valid:
        # --- proceed as your original code path ---
        tri_indices = ear_clip_triangulate(poly2)
        if len(tri_indices) == 0:
            raise ValueError("Triangulation failed. Check polygon geometry.")
        min_x = int(math.floor(min(x for x, z in poly2)))
        max_x = int(math.ceil(max(x for x, z in poly2)))
        min_z = int(math.floor(min(z for x, z in poly2)))
        max_z = int(math.ceil(max(z for x, z in poly2)))
        filled_blocks = []
        missed = 0
        for X in range(min_x, max_x + 1):
            for Z in range(min_z, max_z + 1):
                sample = (X + 0.5, Z + 0.5)
                if not poly_shape.contains(Point(sample)):
                    continue
                assigned = False
                for (i, j, k) in tri_indices:
                    a2, b2, c2 = poly2[i], poly2[j], poly2[k]
                    bary = point_in_triangle_bary(a2, b2, c2, sample)
                    if bary is None:
                        continue
                    u, v, w = bary
                    if (u >= -1e-9) and (v >= -1e-9) and (w >= -1e-9):
                        ya, yb, yc = poly3[i][1], poly3[j][1], poly3[k][1]
                        y = u * ya + v * yb + w * yc
                        filled_blocks.append((X, int(round(y)), Z))
                        assigned = True
                        break
                if not assigned:
                    best = None
                    best_minw = -1.0
                    for (i, j, k) in tri_indices:
                        a2, b2, c2 = poly2[i], poly2[j], poly2[k]
                        bary = point_in_triangle_bary(a2, b2, c2, sample)
                        if bary is None:
                            continue
                        u, v, w = bary
                        minw = min(u, v, w)
                        if minw > best_minw:
                            best_minw = minw
                            best = (i, j, k, bary)
                    if best is not None:
                        i, j, k, bary = best
                        u, v, w = bary
                        ya, yb, yc = poly3[i][1], poly3[j][1], poly3[k][1]
                        y = u * ya + v * yb + w * yc
                        filled_blocks.append((X, int(round(y)), Z))
                    else:
                        avg_y = sum(p[1] for p in poly3) / len(poly3)
                        filled_blocks.append((X, int(round(avg_y)), Z))
                        missed += 1

        # Edge blocks from original outline
        edge_blocks = []
        for i in range(len(poly3)):
            x0, y0, z0 = poly3[i]
            x1, y1, z1 = poly3[(i + 1) % len(poly3)]
            edge_points = bresenham_line(int(round(x0)), int(round(z0)),
                                         int(round(x1)), int(round(z1)))
            n = len(edge_points)
            for idx, (x, z) in enumerate(edge_points):
                t = idx / max(1, n - 1)
                y = (1 - t) * y0 + t * y1
                edge_blocks.append((x, int(round(y)), z))

        return set(filled_blocks) | set(edge_blocks)

    # ---------- self-intersecting path handling ----------
    # Build shapely segments from the original ring
    segs_with_y = build_segments_with_y(poly3)
    line_segs = [LineString([(x1, z1), (x2, z2)])
                 for (x1, z1, _y1, x2, z2, _y2) in segs_with_y]

    # Merge and polygonize into simple faces
    merged = unary_union(line_segs)
    faces = list(polygonize(merged))  # list of simple Polygons

    # Keep only faces that are inside the original ring by even-odd rule
    kept_faces = []
    for face in faces:
        c = face.representative_point()  # robust interior point
        if ray_cast_even_odd((c.x, c.y), poly2):
            kept_faces.append(face)

    if not kept_faces:
        # As a last resort, bail out rather than crash
        raise ValueError("Self-intersecting outline produced no fillable faces.")

    all_filled = []
    all_edges = []

    # Process each kept face: reconstruct 3D boundary (interpolate Y at new vertices), triangulate, fill
    for face in kept_faces:
        coords2d = list(face.exterior.coords)
        # Drop closing duplicate
        if len(coords2d) > 1 and coords2d[0] == coords2d[-1]:
            coords2d = coords2d[:-1]

        # Build 3D boundary by interpolating Y along the original segments for each vertex
        face3 = []
        for (x, z) in coords2d:
            y = interpolate_y_at_point(x, z, segs_with_y, tol=1e-6)
            face3.append((x, y, z))

        # Triangulate in 2D (XZ) and fill like before
        face2 = [(x, z) for (x, y, z) in face3]
        tri_indices = ear_clip_triangulate(face2)
        if not tri_indices:
            continue  # skip degenerate face

        min_x = int(math.floor(min(x for x, z in face2)))
        max_x = int(math.ceil(max(x for x, z in face2)))
        min_z = int(math.floor(min(z for x, z in face2)))
        max_z = int(math.ceil(max(z for x, z in face2)))
        face_shape = Polygon(face2)

        for X in range(min_x, max_x + 1):
            for Z in range(min_z, max_z + 1):
                sample = (X + 0.5, Z + 0.5)
                if not face_shape.contains(Point(sample)):
                    continue
                placed = False
                for (i, j, k) in tri_indices:
                    a2, b2, c2 = face2[i], face2[j], face2[k]
                    bary = point_in_triangle_bary(a2, b2, c2, sample)
                    if bary is None:
                        continue
                    u, v, w = bary
                    if (u >= -1e-9) and (v >= -1e-9) and (w >= -1e-9):
                        ya, yb, yc = face3[i][1], face3[j][1], face3[k][1]
                        y = u * ya + v * yb + w * yc
                        all_filled.append((X, int(round(y)), Z))
                        placed = True
                        break
                if not placed:
                    # Closest-bary fallback (rare)
                    best = None
                    best_minw = -1.0
                    for (i, j, k) in tri_indices:
                        a2, b2, c2 = face2[i], face2[j], face2[k]
                        bary = point_in_triangle_bary(a2, b2, c2, sample)
                        if bary is None:
                            continue
                        u, v, w = bary
                        minw = min(u, v, w)
                        if minw > best_minw:
                            best_minw = minw
                            best = (i, j, k, bary)
                    if best is not None:
                        i, j, k, bary = best
                        u, v, w = bary
                        ya, yb, yc = face3[i][1], face3[j][1], face3[k][1]
                        y = u * ya + v * yb + w * yc
                        all_filled.append((X, int(round(y)), Z))

        # Edge rasterization for this face boundary (so outlines include intersection splits)
        for i in range(len(face3)):
            x0, y0, z0 = face3[i]
            x1, y1, z1 = face3[(i + 1) % len(face3)]
            edge_points = bresenham_line(int(round(x0)), int(round(z0)),
                                         int(round(x1)), int(round(z1)))
            n = len(edge_points)
            for idx, (x, z) in enumerate(edge_points):
                t = idx / max(1, n - 1)
                y = (1 - t) * y0 + t * y1
                all_edges.append((x, int(round(y)), z))

    return set(all_filled) | set(all_edges)

def clean_coords(coords):
    cleaned = []
    for pt in coords:
        if not cleaned or pt != cleaned[-1]:  # remove consecutive duplicates
            cleaned.append(pt)
    if len(cleaned) > 1 and cleaned[0] == cleaned[-1]:
        cleaned = cleaned[:-1]  # drop closing duplicate
    return cleaned

## Create buildings JSON

Load cropped GeoJson

In [5]:
layer = "fkb_bygning_grense"
input_data_path = f"cropped/{layer}-CROPPED.geojson"

data = geopandas.read_file(filename=input_data_path)

Loop through data and populate chunk dict.

In [7]:
priority_list = {
    "Takkant": 1,
    "Taksprang": 1,
    "TakoverbyggKant": 1,
    "FiktivBygningsavgrensning": 2,
    "Arkade": 3,
    "Portrom": 3,
    "Veranda": 4,
    "Låvebru": 5,
    "TrappBygg": 5,
    "VeggFrittstående": 6,
    "Bygningsdelelinje": 7,
    "TaksprangBunn": 7,
    "Bygningslinje": 8,
    "Mønelinje": 9,
    "TakplatåTopp": 10,
    "Takplatå": 11,
    "BygningBru": 12
}

discard_list = ["AnnenBygning", "Bygning", "Fasadeliv", "Grunnmur", "Hjelpelinje3D", "TakMur", "Takoverbygg", "BygningsavgrensningTiltak"]

regions = {}

for index, row in data.iterrows():
    if row["objtype"] in discard_list:
        continue
    s = geopandas.GeoSeries(row['geometry'])
    coordinates = s.get_coordinates(index_parts=True, include_z=True)
    all_blocks: List[Tuple[int, int, int]] = []
    start: List[float] = []
    end: List[float] = []
    for i, coord_pair in coordinates.iterrows():
        if i == (0, 0):
            start = coord_pair.tolist()
            continue
        end = coord_pair.tolist()

        start_block_x, start_block_z = from_geo(start[1], start[0])
        end_block_x, end_block_z = from_geo(end[1], end[0])

        blocks = bresenham3d(int(math.floor(start_block_x)), int(math.floor(start[2])), int(math.floor(start_block_z)),
                             int(math.floor(end_block_x)), int(math.floor(end[2])), int(math.floor(end_block_z)))

        all_blocks += blocks

        start = coord_pair.tolist()

    unique_blocks = list(set(all_blocks))

    for block in unique_blocks:
        region_key = f"{block[0]//512}_{block[2]//512}"
        chunk_key = f"{block[0]//16}_{block[2]//16}"
        if region_key not in regions:
            regions[region_key] = {}
        if chunk_key not in regions[region_key]:
            regions[region_key][chunk_key] = []
        regions[region_key][chunk_key].append((block[0],
                            block[1],
                            block[2],
                            priority_list[row["objtype"]]
                            ))

Write chunk dict to Json

In [8]:
'''
with open("output.json", "w") as json_file:
    json.dump(chunks, json_file, indent=4)  # indent=4 for pretty-printing
'''

for key, value in regions.items():
    with open("buildings/"+key+".json", 'w') as f:
        json.dump(value, f, indent=4)

## Create road JSON

In [5]:
'''
bergen_veger_path = "/Volumes/TOSHIBA EXT/data/bergenveger.geojson"
bergen_grense_path = "/Volumes/TOSHIBA EXT/data/bergenvegergrense.geojson"
bergen_veg_senterlinje_path = "/Volumes/TOSHIBA EXT/data/bergenvegersenterlinje.geojson"

bergen_veger_data = geopandas.read_file(filename=bergen_veger_path)
bergen_grense_data = geopandas.read_file(filename=bergen_grense_path)
bergen_veg_senterlinje_data = geopandas.read_file(filename=bergen_veg_senterlinje_path)

all_data = [bergen_veger_data, bergen_grense_data, bergen_veg_senterlinje_data]
'''

layers = ["fkb_veg_senterlinje", "fkb_veg_grense", "fkb_veg_område"]
input_file_paths = [f"cropped/{layer}-CROPPED.geojson" for layer in layers]

all_data = [geopandas.read_file(filename=file_path) for file_path in input_file_paths]

In [ ]:
line_priority = {
    "Vegdekkekant": 1,
    "Vegskulderkant": 2,
    "Vegrekkverk": 12,
    "AnnetVegarealAvgrensning": 13,
    "OverkjørbartArealAvgrensning": 14,
    "Vegbom": 15,
    "Skiltportal": 16,
    "Kantstein": 17,
    "Kjørebanekant": 18,
    "Vegoppmerking": 18
}
line_priority = {key.lower(): value for key, value in line_priority.items()}

area_priority = {
    "VegKjørende": 3,
    "VegGåendeOgSyklende": 4,
    "ParkeringsOmråde": 5,
    "Kjørebanekant": 6,
    "Vegdekkekant": 6,
    "Vegskulderkant": 6,
    "Trafikkøy": 7,
    "FartsdemperAvgrensning": 8,
    "GangfeltAvgrensning": 9,
    "FeristAvgrensning": 10,
    "AnnetVegarealAvgrensning": 11,
    "Vegrekkverk": 12,
    "Vegoppmerking": 18
}

area_priority = {key.lower(): value for key, value in area_priority.items()}

all_priorities = {}

for key, val in line_priority.items():
    all_priorities[(key, "line")] = val

for key, val in area_priority.items():
    all_priorities[(key, "area")] = val

line_to_polygon = ["Fartsdemperavgrensning", "GangfeltAvgrensning", "FeristAvgrensning"]
line_to_polygon = [value.lower() for value in line_to_polygon]

discard_list = ["VegAnnenAvgrensning", "VegFiktivGrense"]
discard_list = [value.lower() for value in discard_list]

regions = {}

total_rows = sum(len(df) for df in all_data)
processed = 0

for data in all_data:
    for index, row in data.iterrows():
        objtype = row["objtype"].lower()
        if objtype in discard_list:
            continue
        geom = row['geometry']
        geom_type = geom.geom_type.upper()

        all_blocks: List[Tuple[int, int, int]] = []

        p = None

        coords_count = 0
        if geom_type == "LINESTRING":
            coords_count = len(list(geom.coords))
        elif geom_type == "MULTILINESTRING":
            coords_count = sum(len(list(line.coords)) for line in geom.geoms)
        elif geom_type == "POLYGON":
            coords_count = len(list(geom.exterior.coords))
        elif geom_type == "MULTIPOLYGON":
            coords_count = sum(len(list(poly.exterior.coords)) for poly in geom.geoms)

        if (line_priority.get(objtype, None) is not None and geom_type in ["LINESTRING", "MULTILINESTRING"]) or (
                objtype in line_to_polygon and coords_count <= 2
        ):

            p = line_priority.get(objtype) or area_priority.get(objtype)

            coordinates = []
            if geom.geom_type == "LineString":
                coordinates = list(geom.coords)
            elif geom.geom_type == "MultiLineString":
                for line in geom.geoms:
                    coordinates.extend(list(line.coords))

            start = None
            for coord in coordinates:
                if start is None:
                    start = coord
                    continue
                end = coord

                start_block_x, start_block_z = from_geo(start[1], start[0])
                end_block_x, end_block_z = from_geo(end[1], end[0])

                blocks = bresenham3d(
                    int(math.floor(start_block_x)), int(math.floor(start[2] if len(start) > 2 else 0)),
                    int(math.floor(start_block_z)),
                    int(math.floor(end_block_x)), int(math.floor(end[2] if len(end) > 2 else 0)),
                    int(math.floor(end_block_z))
                )

                all_blocks += blocks
                start = end


        elif area_priority.get(objtype, None) is not None and geom_type == "MULTIPOLYGON" or (

                objtype in line_to_polygon and (

                geom_type in ["POLYGON", "MULTIPOLYGON"] or geom_type in ["LINESTRING", "MULTILINESTRING"]

        )):

            p = area_priority.get(objtype) or line_priority.get(objtype)

            polygons = []

            if geom.geom_type == "Polygon":
                polygons = [geom]

            elif geom.geom_type == "MultiPolygon":
                polygons = list(geom.geoms)

            elif geom.geom_type in ["LineString", "MultiLineString"] and objtype in line_to_polygon:
                # Convert lines to polygons
                polygons = list(polygonize(geom))
                
            else:
                raise Exception(f"Unsupported geometry type for area: {geom.geom_type}")

            all_blocks = []

            for poly in polygons:

                if not poly.is_valid or poly.is_empty:
                    continue

                # --- Exterior ---

                exterior_coords = []

                for x, y, *rest in poly.exterior.coords:
                    z = rest[0] if rest else 0
                    bx, bz = from_geo(y, x)
                    exterior_coords.append((bx, z, bz))

                exterior_coords = clean_coords(exterior_coords)

                if len(exterior_coords) >= 4:
                    points = get_polygon_points(exterior_coords)

                    all_blocks += list(points)

                # --- Interiors (holes) ---

                for interior in poly.interiors:
                    interior_coords = []

                    for x, y, *rest in interior.coords:
                        z = rest[0] if rest else 0
                        bx, bz = from_geo(y, x)
                        interior_coords.append((bx, z, bz))
                        
                    interior_coords = clean_coords(interior_coords)

                    if len(interior_coords) >= 4:
                        hole_points = get_polygon_points(interior_coords)

                        # subtract holes: compare (x, z) only
                        hole_points_no_y = [[ht[0], ht[2]] for ht in hole_points]
                        all_blocks = [pt for pt in all_blocks if [pt[0], pt[2]] not in hole_points_no_y]

        else:
                raise(Exception("Priority type not found", objtype, geom_type))
                p = None


        # filtered_dict = {}
        # p = all_priorities[objtype]
        # for x, y, z in all_blocks:
        #     coords = (x, y, z)
        #     if coords not in filtered_dict or p > filtered_dict[coords][3]:
        #         filtered_dict[coords] = (x, y, z, p)

        unique_blocks = list(set(all_blocks))
        #print("unique blocks", len(unique_blocks))

        for block in unique_blocks:
            bx, by, bz = block
            region_key = f"{bx//512}_{bz//512}"
            chunk_key = f"{bx//16}_{bz//16}"

            if region_key not in regions:
                regions[region_key] = {}
            if chunk_key not in regions[region_key]:
                regions[region_key][chunk_key] = {}

            coords = (bx, by, bz)
            # keep only the highest priority block at this position
            if coords not in regions[region_key][chunk_key] or (p > regions[region_key][chunk_key][coords][3]):
                regions[region_key][chunk_key][coords] = (bx, by, bz, p)

In [ ]:
# Convert back into the plugin's expected JSON structure
region_output = {}
for region_key, region in regions.items():
    output_chunks = {}
    for chunk_key, coordmap in region.items():
        output_chunks[chunk_key] = list(coordmap.values())
    region_output[region_key] = output_chunks

In [ ]:
# Write to JSON
for key, value in region_output.items():
    with open("roads/"+key+".json", 'w') as f:
        json.dump(value, f, indent=4)